Reproduction Notes:
- use Python 3.7 to run the code
- remove dirs_exist_ok=True in realtabformer.py



In [1]:
# pip install realtabformer
import os
import pandas as pd
from pathlib import Path
from realtabformer import REaLTabFormer

/home/arthur/miniforge3/envs/cs848/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load the data
parent_df = pd.read_csv("../rossmann-data/raw/store.csv")
child_df = pd.concat([pd.read_csv("../rossmann-data/raw/train.csv"), 
                      pd.read_csv("../rossmann-data/raw/test.csv")], 
                      ignore_index=True)
join_on = "Store"

# Make sure that the key columns in both the parent and the child table have the same name.
assert ((join_on in parent_df.columns) and
        (join_on in child_df.columns))

/home/arthur/miniforge3/envs/cs848/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Non-relational or parent table. Don't include the unique_id field.
parent_model = REaLTabFormer(model_type="tabular")
parent_model.fit(parent_df.drop(join_on, axis=1))

pdir = Path("rtf_parent/")
parent_model.save(pdir)

# # Get the most recently saved parent model,
# # or a specify some other saved model.
# parent_model_path = pdir / "idXXX"
parent_model_path = sorted([
    p for p in pdir.glob("id*") if p.is_dir()],
    key=os.path.getmtime)[-1]

/home/arthur/miniforge3/envs/cs848/lib/python3.7/site-packages/realtabformer/realtabformer.py:598: UserWarning: qt_interval adjusted from 100 to 9...
  f"qt_interval adjusted from {qt_interval} to {_qt_interval}..."


Computing the sensitivity threshold...
Using parallel computation!!!


Bootstrap round:  31%|███       | 156/500 [00:01<00:02, 152.60it/s]

KeyboardInterrupt: 

Bootstrap round:  31%|███       | 156/500 [00:19<00:02, 152.60it/s]

In [ ]:
# load the child model
child_model = REaLTabFormer(
    model_type="relational",
    parent_realtabformer_path=parent_model_path,
    output_max_length=None,
    train_size=0.8)

child_model.fit(
    df=child_df,
    in_df=parent_df,
    join_on=join_on)

In [ ]:
# Generate parent samples.
parent_samples = parent_model.sample(len(parent_df))

# Create the unique ids based on the index.
parent_samples.index.name = join_on
parent_samples = parent_samples.reset_index()

# Generate the relational observations.
child_samples = child_model.sample(
    input_unique_ids=parent_samples[join_on],
    input_df=parent_samples.drop(join_on, axis=1),
    gen_batch=64)